In [14]:
import numpy as np
import scipy as sci
import sympy as sy
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

## Jacobi

In [2]:
def GetacobiMethod(A,b,itmax=1000,error = 1e-10):
    
    M,N = A.shape
    
    x = np.zeros(N)
    
    sumk = np.zeros_like(x)
    
    x = [13,20,-1]
    
    it = 0
    
    residuo = np.linalg.norm( b - np.dot(A,x) )
    
    while ( residuo > error and it < itmax ):
        
        it += 1
        
        for i in range(M):
            sum_ = 0
            for j in range(N):
                if i != j:
                    sum_ += A[i][j]*x[j]
            sumk[i] = sum_
          
        for i in range(M):
            
            if A[i,i] != 0:
                x[i] = (b[i] - sumk[i])/A[i,i]
            else:
                print('No invertible con Jacobi')
                
        residuo = np.linalg.norm( b - np.dot(A,x) )
        
    return x,it,residuo


In [6]:
def GetT(A):
    
    D = np.zeros_like(A)
    R = np.zeros_like(A)
    
    n = A.shape[0]
    
    for i in range(n):
        for j in range(n):
            if i == j:
                D[i,j] = 1/A[i,j]
            else:
                R[i,j] = A[i,j]
                
    return np.dot(D,R)


## NewtonRapson generalizado

In [3]:
def GetVectorF(G,r):
    
    dim = len(G)
    
    v = np.zeros(dim)
    
    for i in range(dim):
        v[i] = G[i](r[0],r[1],r[2])
        
    return v

In [4]:

def GetJacobian(G,r,h=1e-6):
    
    dim = len(G)
    
    J = np.zeros((dim,dim))
    
    for i in range(dim):
        J[i,0] = (  G[i](r[0]+h,r[1],r[2]) - G[i](r[0]-h,r[1],r[2]) )/(2*h)
        J[i,1] = (  G[i](r[0],r[1]+h,r[2]) - G[i](r[0],r[1]-h,r[2]) )/(2*h)
        J[i,2] = (  G[i](r[0],r[1],r[2]+h) - G[i](r[0],r[1],r[2]-h) )/(2*h)
        
    return J.T

In [5]:
def NewtonRaphson(G,r,error=1e-10):
    
    it = 0
    d = 1
    Vector_d = np.array([])
    
    while d > error:
        
        it += 1
        
        rc = r
        
        F = GetVectorF(G,r)
        J = GetJacobian(G,r)
        InvJ = np.linalg.inv(J)
        
        r = rc - np.dot( InvJ, F )
        
        diff = r - rc
        print(diff)
        
        d = np.linalg.norm(diff)
        
        Vector_d = np.append( Vector_d , d )
        
    return r,it,Vector_d

## Gauss Jordan

In [8]:
def Diagonalizar(A_, ones = False):
    
    A = A_.copy()
    n = A.shape[0]
    
    for i in range(n):
        
        if ones:
            A[i,:] /= A[i,i]
        
        
        for j in range(i+1,n):
            
            a = A[j,i]/A[i,i]
            A[j,:] -= a*A[i,:]
            
    return A

In [9]:
def GausJordan(A_,b_,ones = False):
    
    A = A_.copy()
    b = b_.copy()
    
    n = A.shape[0]
    
    for i in range(n):
        
        if ones:
            b[i] /= A[i,i]
            A[i,:] /= A[i,i]
           
        
        for j in range(i+1,n):
            
            a = A[j,i]/A[i,i]
            A[j,:] -= a*A[i,:]
            b[j] -= a*b[i]
         
       
    x = np.zeros_like(b)

    
    for i in reversed(range(n)):
        #print(i)
    
        x[i] = b[i]
        
        for j in range( i+1, n ):
            x[i] -= A[i,j]*x[j]
        
        if not ones:
            x[i] /= A[i,i]
    
    return x,A,b

## Gradiente Descendente no Lineal

In [10]:
def GetVectorF(G,r):
    
    dim = len(G)
    
    v = np.zeros(dim)
    
    for i in range(dim):
        v[i] = G[i](r[0],r[1],r[2])
        
    return v

In [11]:
def GetJacobian(G,r,h=1e-6):
    
    dim = len(G)
    
    J = np.zeros((dim,dim))
    
    for i in range(dim):
        J[i,0] = (  G[i](r[0]+h,r[1],r[2]) - G[i](r[0]-h,r[1],r[2]) )/(2*h)
        J[i,1] = (  G[i](r[0],r[1]+h,r[2]) - G[i](r[0],r[1]-h,r[2]) )/(2*h)
        J[i,2] = (  G[i](r[0],r[1],r[2]+h) - G[i](r[0],r[1],r[2]-h) )/(2*h)
        
    return J.T

In [12]:
def GetMetric(G,r):
    v = GetVectorF(G,r)
    return 0.5*np.linalg.norm(v)**2

In [15]:
def GetFig(F,R,it):
    
    fig = plt.figure(figsize=(8,4))
    
    labels = ['X','Y','Z']
    
    ax = fig.add_subplot(1,2,1)
    ax1 = fig.add_subplot(1,2,2)

    ax.set_title('Metric: %.20f' %(F[it]))

    ax.plot(F[:it])
    ax.set_xlabel('%.0f' %(it))
    ax.set_yscale('log')
    ax1.plot(R[:it],label=labels)
    ax1.set_xlabel('%.0f' %(it))
    ax1.legend(loc=0)
    
    plt.show()

In [16]:
def GetSolve(G,r,lr=1e-3,epochs=int(1e5),error=1e-7):
    
    d = 1
    it = 0
    Vector_F = np.array([])
    
    R_vector = np.array(r)
    
    while d > error and it < epochs:
        
        CurrentF = GetMetric(G,r)
        
        J = GetJacobian(G,r)
        
        GVector = GetVectorF(G,r)
        
        #Machine Learning
        r -= lr*np.dot(J,GVector) 
        
        R_vector = np.vstack((R_vector,r))
        
        NewF = GetMetric(G,r)
        
        
        Vector_F = np.append(Vector_F,NewF)
        
        d = np.abs( CurrentF - NewF )/NewF
        
        
        if it%500 == 0:
            
            #print(it,d)
            clear_output(wait=True)
            GetFig(Vector_F,R_vector,it)
            time.sleep(0.01)
            
        it += 1
        
    if d < error:
        print(' Entrenamiento completo ', d, 'iteraciones', it)
        
    if it == epochs:
        print(' Entrenamiento no completado ')
        
    return r,it,Vector_F,R_vector